In [12]:
import settings
import tweepy
import argparse
import time

import pandas as pd

import get_tweets

import re
import os

import MeCab


In [9]:
df = pd.read_csv('../data/ki_oku/tweet_user_kizuna_100k.csv')

In [11]:
df[:1000]

,Unnamed: 0,user,text
0,0,serugei_anime,@midnight_toyosu お疲れ様でした！！ @FA_010S 万能機…？🤔🤔 @F...
1,1,Akuma25Ren,@kousuke_syma キモオタが出来上がる理由が分かったよ…コミ障でうまく喋れないんだ...
2,2,coil_ganguboy,RT @ARuFa_FARu: チョコシリアルの終盤に飲む「チョコ味の牛乳」が大好きなので、...
3,3,dkndkndkndkn,テイルズオブジアビスに出てきたインディグネイションもエドワード・D・モリスン時代から受け継が...
4,4,nqk13044,@DrD19688276 ちなみに年代的にはどれ位の作品なんです？車がどことなくレトロな感じ...
5,5,Gintoki_kai2,RT @OHAYO_chihaya: ベンチに座っていたらヤンキーにいきなり喧嘩を売られ、ボ...
6,6,_______NOI_____,RT @takanonnotakano: 結婚式の招待状が届いたけど何もかも知らない女のマン...
7,7,otsukimimoon,RT @tortor_08: そうめんに異様な執着をみせるねこ😳 https://t.co/...
8,8,garfunkel_s,RT @yontengoP: なので結論としては、中の人とコンテンツが不可分ってなると、命運...
9,9,___hrks,RT @y_yujirushi: あれは！キズナアイの中でも最も残虐な人格…カズヤ！！ ht...


In [47]:
def text_wakati(text):
    m = MeCab.Tagger('-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd ')
    wa = m.parse(text)
    wa = wa.split(' ')
    return wa

In [48]:
df_kizuna = df[:1000]

In [49]:
df_kizuna['wakati'] = df_kizuna['text'].apply(text_wakati)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
df_kizuna

,Unnamed: 0,user,text,wakati
0,0,serugei_anime,@midnight_toyosu お疲れ様でした！！ @FA_010S 万能機…？🤔🤔 @F...,"[@, midnight, _, toyosu, お疲れ様, でし, た, ！, ！, @,..."
1,1,Akuma25Ren,@kousuke_syma キモオタが出来上がる理由が分かったよ…コミ障でうまく喋れないんだ...,"[@, kousuke, _, syma, キモオタ, が, 出来上がる, 理由, が, 分..."
2,2,coil_ganguboy,RT @ARuFa_FARu: チョコシリアルの終盤に飲む「チョコ味の牛乳」が大好きなので、...,"[RT, @, ARuFa, _, FARu, :, チョコ, シリアル, の, 終盤, に..."
3,3,dkndkndkndkn,テイルズオブジアビスに出てきたインディグネイションもエドワード・D・モリスン時代から受け継が...,"[テイルズオブジアビス, に, 出, て, き, た, インディグネイション, も, エドワ..."
4,4,nqk13044,@DrD19688276 ちなみに年代的にはどれ位の作品なんです？車がどことなくレトロな感じ...,"[@, DrD, 19688276, ちなみに, 年代, 的, に, は, どれ, 位, の..."
5,5,Gintoki_kai2,RT @OHAYO_chihaya: ベンチに座っていたらヤンキーにいきなり喧嘩を売られ、ボ...,"[RT, @, OHAYO, _, chihaya, :, ベンチ, に, 座っ, て, い..."
6,6,_______NOI_____,RT @takanonnotakano: 結婚式の招待状が届いたけど何もかも知らない女のマン...,"[RT, @, takanonnotakano, :, 結婚式, の, 招待, 状, が, ..."
7,7,otsukimimoon,RT @tortor_08: そうめんに異様な執着をみせるねこ😳 https://t.co/...,"[RT, @, tortor, _, 08, :, そうめん, に, 異様, な, 執着, ..."
8,8,garfunkel_s,RT @yontengoP: なので結論としては、中の人とコンテンツが不可分ってなると、命運...,"[RT, @, yontengoP, :, なので, 結論, として, は, 、, 中の人,..."
9,9,___hrks,RT @y_yujirushi: あれは！キズナアイの中でも最も残虐な人格…カズヤ！！ ht...,"[RT, @, y, _, yujirushi, :, あれ, は, ！, キズナアイ, の..."


In [68]:
def vec2dense(vec, num_terms):
    return list(gensim.matutils.corpus2dense([vec], num_terms=num_terms).T[0])

def bagofwords_dct(docs):
    '''
    １文書が単語のリストになった文書群のリスト
    '''
    dct = gensim.corpora.Dictionary(docs)
    unfiltered = dct.token2id.keys()
    # 出現頻度　３未満の単語消す
    # 出現率が0.9以上の単語を消す
    #dct.filter_extremes(no_below=5, no_above=0.9)
    dct.filter_extremes(no_below=400, no_above=0.8)
    filtered = dct.token2id.keys()
    filtered_out = set(unfiltered) - set(filtered)
    print(len(filtered))
    print(len(filtered_out))
    return dct

In [69]:
kizuna_dict = bagofwords_dct(df_kizuna['wakati'].to_list())

455
249347


In [70]:
kizuna_dict.token2id

{'"': 0,
 '&': 1,
 ',': 2,
 '-': 3,
 '00': 4,
 '10': 5,
 '11': 6,
 '12': 7,
 '18': 8,
 ';': 9,
 'D': 10,
 'E': 11,
 'F': 12,
 'G': 13,
 'I': 14,
 'L': 15,
 'M': 16,
 'P': 17,
 'R': 18,
 'S': 19,
 'Twitter': 20,
 'V': 21,
 'Y': 22,
 'YouTube': 23,
 '_:': 24,
 'i': 25,
 'j': 26,
 'l': 27,
 'n': 28,
 'o': 29,
 'p': 30,
 's': 31,
 'u': 32,
 'w': 33,
 'ω': 34,
 '→': 35,
 '✨': 36,
 'あ': 37,
 'あげ': 38,
 'あと': 39,
 'あの': 40,
 'ありがとう': 41,
 'あれ': 42,
 'いえ': 43,
 'いく': 44,
 'いたし': 45,
 'いつも': 46,
 'おめでとう': 47,
 'おり': 48,
 'お前': 49,
 'お願い': 50,
 'かい': 51,
 'かけ': 52,
 'かなり': 53,
 'かわいい': 54,
 'く': 55,
 'くらい': 56,
 'こ': 57,
 'こそ': 58,
 'こちら': 59,
 'こんな': 60,
 'しない': 61,
 'しまっ': 62,
 'しよう': 63,
 'じゃなくて': 64,
 'すぐ': 65,
 'ずっと': 66,
 'ぜ': 67,
 'ぜひ': 68,
 'たち': 69,
 'っていう': 70,
 'つけ': 71,
 'でしょ': 72,
 'とき': 73,
 'とりあえず': 74,
 'どこ': 75,
 'ながら': 76,
 'なん': 77,
 'なんで': 78,
 'にて': 79,
 'ぬ': 80,
 'はず': 81,
 'べき': 82,
 'ほしい': 83,
 'ほど': 84,
 'ま': 85,
 'ましょ': 86,
 'まじ': 87,
 'また': 88,
 'まとめ': 89,
 'め': 90,
 '

In [67]:
kizuna_dict.dfs

{167: 276,
 88: 496,
 528: 201,
 685: 299,
 200: 249,
 628: 285,
 157: 272,
 634: 161,
 130: 277,
 664: 716,
 240: 426,
 672: 648,
 608: 217,
 547: 319,
 160: 365,
 279: 108,
 679: 155,
 158: 397,
 267: 566,
 182: 373,
 298: 145,
 378: 682,
 578: 151,
 650: 293,
 502: 691,
 331: 330,
 326: 141,
 123: 691,
 347: 315,
 397: 291,
 37: 132,
 46: 107,
 52: 205,
 563: 136,
 263: 440,
 334: 212,
 26: 178,
 544: 119,
 618: 310,
 534: 135,
 403: 759,
 400: 478,
 355: 137,
 492: 445,
 593: 239,
 461: 258,
 570: 183,
 546: 120,
 363: 524,
 90: 509,
 565: 187,
 183: 797,
 117: 760,
 521: 649,
 635: 234,
 277: 275,
 552: 353,
 614: 384,
 535: 382,
 27: 232,
 13: 619,
 185: 307,
 15: 446,
 55: 280,
 128: 155,
 562: 183,
 665: 531,
 87: 591,
 616: 233,
 452: 181,
 571: 450,
 411: 167,
 136: 435,
 431: 156,
 622: 455,
 489: 674,
 135: 103,
 164: 546,
 237: 224,
 619: 171,
 149: 134,
 213: 791,
 670: 174,
 600: 247,
 463: 360,
 434: 376,
 214: 661,
 220: 335,
 233: 629,
 491: 177,
 582: 386,
 359: 190,